<a href="https://colab.research.google.com/github/raphaellins/ai-investiments/blob/master/mini_indice_5_2020_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/content/drive/MyDrive/MLAZ/win_5_2020.csv', usecols=['Date', 'Open', 'High', 'Low', 'Close', 'Tick', 'Volume'])

In [ ]:
df.head()

In [ ]:
df.dtypes

1. Analyzing if the candle closed is High or Low and adding a column with this information

In [3]:
df['state'] = df.Open < df.Close

In [ ]:
df.head()

2. Adding the info about the candle body size and shadow low and shadow high (Percentage). The entire candle is 100% including the shadows. 

In [4]:
# candle size = High - Low


df['candle_size'] = df.High - df.Low
df['body_size'] = abs(df.Open - df.Close)
df['shade_low'] = np.where(df['state'] == True, (df.Open - df.Low), (df.Close - df.Low))
df['shade_high'] = np.where(df['state'] == True, (df.High - df.Close), (df.High - df.Open))


In [ ]:
df.head()

,Date,Open,High,Low,Close,Tick,Volume,state,body_size,shade_low,shade_high,candle_size
0,2020.01.02 09:00,118940,119042,118889,119016,12658,61538,True,76,51,26,153
1,2020.01.02 09:05,119011,119113,118915,119098,16174,72643,True,87,96,15,198
2,2020.01.02 09:10,119098,119123,119016,119021,12359,49335,False,77,5,25,107
3,2020.01.02 09:15,119021,119042,118981,118996,9840,39651,False,25,15,21,61
4,2020.01.02 09:20,119001,119016,118965,119001,7253,32205,False,0,36,15,51


In [ ]:
df_limit = df.iloc[0:50]

fig = go.Figure(data=[go.Candlestick(x=df_limit['Date'],
                open=df_limit['Open'],
                high=df_limit['High'],
                low=df_limit['Low'],
                close=df_limit['Close'])])

fig.show()

3. Let`s identify the candle with some Price Action Patterns

In [5]:
df['doji'] = df.body_size < df.candle_size * 0.03
df['hammer'] = np.where(df.shade_low > df.body_size * 2, np.where(df.shade_high < df.body_size * 0.1, True, False), False)
df['inverted_hammer'] = np.where(df.shade_low < df.body_size*0.1, np.where(df.shade_high > df.body_size * 2, True, False), False) 

In [6]:
for i in range(1, len(df)):
  if (i + 1) < len(df):
    df.loc[i, 'next_state'] = df.loc[i+1, 'state']

4. Extract the colums related with the candle size and lets try to use some processing model on that

In [7]:
dfc = df[['state', 'body_size', 'shade_low', 'shade_high', 'candle_size', 'Volume', 'next_state']]

In [ ]:
dfc.dropna(inplace=True)

In [ ]:
dfc['bull'] = np.where(dfc['state'] == True, 1, 0)
dfc['next_bull'] =  np.where(dfc['next_state'] == True, 1, 0)

dfc.drop(dfc.columns[[0, 6]], axis=1, inplace=True)

In [ ]:
dfc["body_size_pct"] = round((dfc["body_size"] / dfc["candle_size"]) * 100, 2)
dfc["shade_low_pct"] = round((dfc["shade_low"] / dfc["candle_size"]) * 100, 2)
dfc["shade_high_pct"] = round((dfc["shade_high"] / dfc["candle_size"]) * 100, 2)

In [32]:
dfc

,body_size,shade_low,shade_high,candle_size,Volume,bull,next_bull,body_size_pct,shade_low_pct,shade_high_pct
1,87,96,15,198,72643,1,0,43.94,48.48,7.58
2,77,5,25,107,49335,0,0,71.96,4.67,23.36
3,25,15,21,61,39651,0,0,40.98,24.59,34.43
4,0,36,15,51,32205,0,1,0.00,70.59,29.41
5,56,15,15,86,33175,1,0,65.12,17.44,17.44
...,...,...,...,...,...,...,...,...,...,...
26943,345,0,0,345,89610,0,1,100.00,0.00,0.00
26944,65,10,55,130,48448,1,0,50.00,7.69,42.31
26945,35,65,25,125,34037,0,0,28.00,52.00,20.00
26946,30,20,105,155,37864,0,0,19.35,12.90,67.74
